## Simple regression

Given 1 independent variable and 1 dependent variable, estimate their relationship with a linear function.

### Example problem: house prices

We have data about sold houses: area and selling price.
We want to predict the selling price of our house (dependent variable).
We know its area (independent variable).

Generate random data

In [24]:
import random

area, price = [], []
for _ in range(50):
    a = random.randint(30, 120)
    area.append(a)
    p = (a + 1) * (850_000 + random.randint(-100_000, 200_000))
    if a > 90:
        p = (a + 20) * (450_000 + random.randint(-100_000, 200_000))
    price.append(p)

Plot data

In [26]:
import plotly.express as px

fig = px.scatter(x=area, y=price, labels={"x": "Area [m²]", "y": "Price [Ft]"})
fig.show()

### Problem: find a *line* that best fits the data

Line ~ linear function: $y = ax + b$

We need to find values for the parameters $a$ and $b$.

In [27]:
a = 900_000
b = 0
x_range = [min(area), max(area)]
fig2 = px.line(x=x_range, y=[a * x + b for x in x_range])
fig2.add_traces(fig.data)
fig2.show()

#### Measure the distance from the data (the error, or loss)

Mean Squared Error

$$MSE = \frac{1}{m} \sum_{i=1}^{m} (ax_i+b - y_i)^2$$

($m$ is the number of elements in the teaching data, $n$ is used for the number of input variables, which is 1 now)

In [28]:
def distance(a, b, x, y):
    return (a * x + b - y) ** 2


def error(a, b):
    return sum([distance(a, b, x, y) for x, y in zip(area, price)]) / len(area)


f"{error(a, b):_}"

'354_435_177_821_166.25'

### Minimize the error

Find values for $a$ and $b$ that minimize the error function.

This is an optimization problem, where $a$ and $b$ are the decision variables, and the objective is to minimize $MSE(a,b)$.

In [29]:
# try manually
a = 900_000
b = -100_000
error(a, b) / 10_000_000_000_000

35.252880924116624

In [30]:
# brute-force search
best_a, best_b = a, b
best = error(a, b)
import numpy as np

for a, b in zip(
    np.linspace(800_000, 1_000_000, 10_000), np.linspace(-2_500_000, 0, 10_000)
):
    err = error(a, b)
    if err < best:
        best = err
        best_a, best_b = a, b
f"{best_a=:_}, {best_b=:_}, best={best / 10_000_000_000_000}"

'best_a=800_000.0, best_b=-2_500_000.0, best=19.597696952116625'

To better understand the problem, look at the plot of the error function for different $a,b$ values.

In [31]:
# plot error(a,b)
import plotly.graph_objects as go
import numpy as np

a_range = np.linspace(800_000, 1_000_000, 100)
b_range = np.linspace(-1_000_000, 3_000_000, 100)
err_values = [[error(a, b) for a in a_range] for b in b_range]
fig3 = go.Figure(data=[go.Surface(x=a_range, y=b_range, z=err_values)])
# fig3.write_html("3dplot.html")
fig3.show()

#### We can use a smarter search method than brute-force.

LP and MILP are out of the question, as the objective function is not linear.

We could use a genetic algorithm or other sophisticated metaheuristic but it would be overkill. It's just a quadratic function, only with 2 variables.

If it had only 1 variable, we could easily solve its derivative equation for 0. With multiple variables it becomes a lot more complicated but still possible, see [the normal equation method](https://www.datacamp.com/tutorial/tutorial-normal-equation-for-linear-regression).

Instead of this, we will use a simpler, iterative numerical method, the **gradient descent**. It uses the partial derivatives to guide the search in the right direction, so we need much fewer steps than for brute-force search. This method will be useful in other machine learning problems as well, while the normal equation method is only for linear regression.

The partial derivatives at any point give us the steepness of the slope (the gradient) at that place. If it's high, we are far from the bottom.

$$MSE(a,b) = \frac{1}{m} \sum_{i=1}^{m} (ax_i+b - y_i)^2$$

$$\frac{\partial MSE(a,b)}{\partial a} = \frac{2}{m} \sum_{i=1}^{m} x_i(ax_i+b - y_i)$$

$$\frac{\partial MSE(a,b)}{\partial b} = \frac{2}{m} \sum_{i=1}^{m} (ax_i+b - y_i)$$

*The constant factor $\frac{2}{m}$ is irrelevant for optimization. The 2 is often omitted but the $\frac{1}{m}$ is good for normalization.*

Start from any $(a,b)$ point and update the position based on the gradient.

*If the gradient is positive, we have to decrease the variable, if it's negative, we have to increase.*

$$a_{k+1} = a_k - \alpha \frac{\partial MSE(a,b)}{\partial a}$$

$$b_{k+1} = b_k - \alpha \frac{\partial MSE(a,b)}{\partial b}$$

$\alpha$ is the learning rate, or step size parameter. If it's too small, convergence will be slow but if it's too large, it may even diverge.

![](https://miro.medium.com/max/626/1*GjLAgjRNpbbwJPtRdmmh7w.png)


In [32]:
# use numpy arrays for simpler and faster operations
x = np.array(area, dtype=np.float64)
y = np.array(price, dtype=np.float64)
a = 800_000
b = 100_000
alpha = 1e-6
alpha1 = 1e-6
alpha2 = 1e-2
step_limit = 4000
for step in range(step_limit):
    f = a * x + b - y  # the common part
    f /= len(area)
    a -= alpha1 * np.dot(x, f)
    b -= alpha2 * np.sum(f)
    print(
        f"Step {step+1:04}: {int(a):,} Ft/m² {int(b):+,} Ft, error: {error(a,b)/100_000_000_000}"
    )

Step 0001: 799,545 Ft/m² +77,821 Ft, error: 2002.4173034293244
Step 0002: 799,095 Ft/m² +56,204 Ft, error: 1997.4683029789392
Step 0003: 798,650 Ft/m² +35,138 Ft, error: 1992.6464164190324
Step 0004: 798,209 Ft/m² +14,616 Ft, error: 1987.9476651225991
Step 0005: 797,773 Ft/m² -5,372 Ft, error: 1983.3681959866256
Step 0006: 797,341 Ft/m² -24,834 Ft, error: 1978.9042774707525
Step 0007: 796,913 Ft/m² -43,780 Ft, error: 1974.5522957608969
Step 0008: 796,489 Ft/m² -62,216 Ft, error: 1970.3087510539954
Step 0009: 796,069 Ft/m² -80,152 Ft, error: 1966.1702539599642
Step 0010: 795,654 Ft/m² -97,596 Ft, error: 1962.1335220172311
Step 0011: 795,242 Ft/m² -114,555 Ft, error: 1958.1953763182116
Step 0012: 794,835 Ft/m² -131,037 Ft, error: 1954.352738241307
Step 0013: 794,431 Ft/m² -147,050 Ft, error: 1950.6026262860162
Step 0014: 794,031 Ft/m² -162,602 Ft, error: 1946.9421530079353
Step 0015: 793,635 Ft/m² -177,699 Ft, error: 1943.3685220504963
Step 0016: 793,242 Ft/m² -192,350 Ft, error: 1939.87

In [33]:
x_range = [min(area), max(area)]
fig4 = px.line(x=x_range, y=[a * x + b for x in x_range])
fig4.add_traces(fig.data)
fig4.show()

### Off-the-shelf solutions

In [34]:
from scipy import stats

result = stats.linregress(area, price)
print(result)
slope, intercept, r, p, std_err = result
# plot
x_range = np.linspace(min(area), max(area), 100)
trace = go.Scatter(
    x=x_range,
    y=[slope * x + intercept for x in x_range],
    mode="lines",
    showlegend=False,
)
trace.line["color"] = "red"
fig5 = go.Figure(data=(trace,) + fig4.data)
fig5.show()

LinregressResult(slope=479683.4804592968, intercept=21790582.11851809, rvalue=0.8170916135071871, pvalue=4.583654255657513e-13, stderr=48850.45486446517, intercept_stderr=3930249.0698368335)


In [40]:
# using numpy.polynomial
from numpy.polynomial import Polynomial

b, a = Polynomial.fit(area, price, 1).convert()
print(f"{a=:_} {b=:_}")
trace = go.Scatter(
    x=x_range, y=[a * x + b for x in x_range], mode="lines", showlegend=False
)
trace.line["color"] = "orange"
fig6 = go.Figure(data=(trace,) + fig4.data)
fig6.show()

a=479_683.48045929713 b=21_790_582.11851807


In [41]:
# approximate with quadratic polynomial
c, b, a = Polynomial.fit(area, price, 2).convert()
print(f"{a=:_} {b=:_} {c=:_}")
trace = go.Scatter(
    x=x_range,
    y=[a * x * x + b * x + c for x in x_range],
    mode="lines",
    showlegend=False,
)
trace.line["color"] = "red"
fig7 = go.Figure(data=(trace,) + fig4.data)
fig7.show()

a=-8_631.678523361405 b=1_817_352.0110138934 c=-22_180_487.18749953


In [44]:
# approximate with cubic polynomial
factors = Polynomial.fit(area, price, 3).convert()
print(f"{factors}")
# Coefficient `a` is very small compared to the others, so it is similar to the quadratic
trace = go.Scatter(
    x=x_range,
    y=[a * x**3 + b * x**2 + c * x + d for x in x_range],
    mode="lines",
    showlegend=False,
)
trace.line["color"] = "red"
fig7 = go.Figure(data=(trace,) + fig4.data)
fig7.show()
# Higher order polynomials can fit the training set more closely but watch out for overfitting!

-7726860.8830838 + 1137097.76247377 x + 1043.64027003 x**2 -
42.233733 x**3


Use the `statsmodel` integration in Plotly Express to draw a trendline.

In [38]:
# use the statsmodels module integration
ols_fig = px.scatter(
    # OLS = Ordinary Least Squares
    x=area,
    y=price,
    labels={"x": "Area [m²]", "y": "Price [Ft]"},
    trendline="ols",
)
ols_fig.show()

[LOWESS](https://en.wikipedia.org/wiki/Local_regression) trendline (LOcally WEighted Scatterplot Smoothing)

In [45]:
lowess_fig = px.scatter(
    x=area, y=price, labels={"x": "Area [m²]", "y": "Price [Ft]"}, trendline="lowess"
)
lowess_fig.show()